In [1]:
import pandas as pd
from datetime import timedelta
#Data processing
from sklearn import preprocessing

In [2]:
df = pd.read_parquet('../../Sonar/seatunnel_all_information.parquet')

In [3]:
df['time'] = df['total_time'].astype('int64')

In [4]:
X = df.loc[:, ~df.columns.isin(['begin_sha', 'end_sha', 'begin_time', 'end_time', 'total_time', 'time'])]

In [5]:
X.head(1)

,commits,additions,deletions,changed_files,begin_Dispensables,begin_Bloaters,begin_Change Preventers,begin_Couplers,begin_Object-Orientation Abusers,end_Dispensables,end_Bloaters,end_Change Preventers,end_Couplers,end_Object-Orientation Abusers
0,1,10,4,3,500.0,262.0,262.0,304.0,304.0,500.0,261.0,261.0,304.0,693.0


In [96]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import numpy as np
import itertools
from itertools import chain, combinations

# Step 1: Prepare the data set
# data = X

# Step 2: Use MinMaxScaler
# scaler = MinMaxScaler()
# scaled_data = scaler.fit_transform(data)

# Step 6: Loop through combinations
lst = X.columns[:14]
combinations_to_check = chain.from_iterable(combinations(lst, r) for r in range(len(lst) + 1))

results = []

for subset in combinations_to_check:

    # Step 1: Prepare the data set
    print("SUBSET :" , subset)

    data = subset

    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(subset)

    # Step 3: Calculate silhouette score for different cluster counts
    silhouette_scores = []
    for n_clusters in range(2, 10):  # You can adjust the range as needed
        kmeans = KMeans(n_clusters=n_clusters)
        cluster_labels = kmeans.fit_predict(scaled_data)
        silhouette_scores.append(silhouette_score(scaled_data, cluster_labels))

    # Step 4: Sort and determine optimal cluster count
    optimal_clusters = silhouette_scores.index(max(silhouette_scores)) + 2
    # Adding 2 due to range start

    # Step 5: Perform KMeans clustering
    kmeans = KMeans(n_clusters=optimal_clusters)
    df['cluster'] = kmeans.fit_predict(scaled_data)

    for cluster_num in range(optimal_clusters):
        sort = cluster_num
        cluster_data = df[df['cluster'] == cluster_num]
        time_min = cluster_data['total_time'].min()
        time_max = cluster_data['total_time'].max()
        time_std = cluster_data['total_time'].std()
        time_minus_2std = cluster_data['total_time'].mean() - 2 * time_std
        time_minus_1std = cluster_data['total_time'].mean() - 1 * time_std
        time_plus_1std = cluster_data['total_time'].mean() + 1 * time_std
        time_plus_2std = cluster_data['total_time'].mean() + 2 * time_std

# # check cluster  1 = 1 in DF
# for cluster_num in range(optimal_clusters):
#     cluster_data = df[df['cluster'] == cluster_num]
#     for subset in combinations_to_check:
#         # Steps 7 and 8: Get time min, time max, and time STDs
#         time_min = cluster_data['total_time'].min()
#         time_max = cluster_data['total_time'].max()
#         time_std = cluster_data['total_time'].std()
#         time_minus_2std = cluster_data['total_time'].mean() - 2 * time_std
#         time_minus_1std = cluster_data['total_time'].mean() - 1 * time_std
#         time_plus_1std = cluster_data['total_time'].mean() + 1 * time_std
#         time_plus_2std = cluster_data['total_time'].mean() + 2 * time_std

        # Step 9: Store results in a dictionary
        result_dict = {
            'combination': subset,
            'cluster_num': cluster_num,
            'silhouette_score': max(silhouette_scores),
            'cluster_sort': optimal_clusters,
            'time_min': time_min,
            'time_max': time_max,
            'time_-2std': time_minus_2std,
            'time_-1std': time_minus_1std,
            'time_+1std': time_plus_1std,
            'time_+2std': time_plus_2std
        }
        results.append(result_dict)

# Create a new DataFrame from the results
results_df = pd.DataFrame(results)

# Display the results
print(results_df)

SUBSET : ()


ValueError: Expected 2D array, got 1D array instead:
array=[].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [90]:
subset

('commits',
 'additions',
 'deletions',
 'changed_files',
 'begin_Dispensables',
 'begin_Bloaters')

In [77]:
time_std

Timedelta('4 days 12:35:56.283149884')

In [78]:
cluster_num

1

In [79]:
optimal_clusters

2

In [80]:
cluster_data

,begin_sha,end_sha,commits,additions,deletions,changed_files,total_time,begin_time,end_time,begin_Dispensables,...,begin_Change Preventers,begin_Couplers,begin_Object-Orientation Abusers,end_Dispensables,end_Bloaters,end_Change Preventers,end_Couplers,end_Object-Orientation Abusers,time,cluster
0,84be0f9fd057b1680d001de38802ce1c28d79f04,9a2efa51c7180b369ecfea17efaae813d9d0bfc5,1,10,4,3,3 days 01:27:48,2023-06-02T07:22:03Z,2023-06-05T08:49:51Z,500.0,...,262.0,304.0,304.0,500.0,261.0,261.0,304.0,693.0,264468000000000,1
1,84be0f9fd057b1680d001de38802ce1c28d79f04,5e03d22d6cb5bfdaf941b7749e0a313bd6ab2f86,1,2,1,1,3 days 04:15:10,2023-06-02T05:07:40Z,2023-06-05T09:22:50Z,500.0,...,262.0,304.0,304.0,501.0,262.0,262.0,304.0,693.0,274510000000000,1
2,84be0f9fd057b1680d001de38802ce1c28d79f04,e0d8519a9d7b6dfac29c1daf4199b6fe87781d0a,1,36,67,9,0 days 05:24:03,2023-06-02T03:16:04Z,2023-06-02T08:40:07Z,500.0,...,262.0,304.0,304.0,499.0,261.0,261.0,304.0,693.0,19443000000000,1
3,84be0f9fd057b1680d001de38802ce1c28d79f04,d0a47b9811a30f5de6d59334bb4fcc8c74ef640b,1,21,2,3,4 days 00:22:26,2023-06-01T08:30:26Z,2023-06-05T08:52:52Z,500.0,...,262.0,304.0,304.0,500.0,261.0,261.0,304.0,693.0,346946000000000,1
4,1b918160214a0b20ce1ae8f9e02f8e0dce243b53,988d2f0c4ef3ba8db0e701ab2105555996def4f7,8,211,212,80,12 days 00:39:27,2023-06-01T07:45:26Z,2023-06-13T08:24:53Z,499.0,...,261.0,304.0,304.0,500.0,262.0,262.0,304.0,693.0,1039167000000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
493,15636bdea1580196118b942d2d3587e0262d2b8f,044f62ef32cd0d195bc4bb6c8492d48d15379ff4,9,412,287,12,1 days 16:30:23,2022-09-28T17:53:03Z,2022-09-30T10:23:26Z,241.0,...,129.0,139.0,139.0,290.0,175.0,175.0,194.0,497.0,145823000000000,1
494,05d105dea3ee53b9d001adbfcf2ec87ea06f8aba,8fd966394b66ef83dca4635a9c278f655903a9bf,16,478,110,19,9 days 11:33:16,2022-09-28T15:15:23Z,2022-10-08T02:48:39Z,290.0,...,175.0,194.0,194.0,290.0,175.0,175.0,194.0,500.0,819196000000000,1
517,41751d8b636064d73ffb66ab07cea0c7e3091e47,acf4d5b1b469adf383b06266ef2e9815f472cd25,15,1601,1,22,110 days 16:44:58,2022-09-21T09:21:12Z,2023-01-10T02:06:10Z,384.0,...,213.0,238.0,238.0,386.0,214.0,214.0,243.0,546.0,9564298000000000,1
524,4969ffae88494b69de694f03a9c534f25dd5d242,38b0daf8b7ba723f2ee18d1e0ab55073701c1b74,39,1164,97,22,32 days 19:01:36,2022-09-18T11:31:28Z,2022-10-21T06:33:04Z,299.0,...,176.0,163.0,163.0,303.0,183.0,183.0,180.0,484.0,2833296000000000,1


In [81]:
time_min

Timedelta('0 days 00:00:06')

In [82]:
cluster_data

,begin_sha,end_sha,commits,additions,deletions,changed_files,total_time,begin_time,end_time,begin_Dispensables,...,begin_Change Preventers,begin_Couplers,begin_Object-Orientation Abusers,end_Dispensables,end_Bloaters,end_Change Preventers,end_Couplers,end_Object-Orientation Abusers,time,cluster
0,84be0f9fd057b1680d001de38802ce1c28d79f04,9a2efa51c7180b369ecfea17efaae813d9d0bfc5,1,10,4,3,3 days 01:27:48,2023-06-02T07:22:03Z,2023-06-05T08:49:51Z,500.0,...,262.0,304.0,304.0,500.0,261.0,261.0,304.0,693.0,264468000000000,1
1,84be0f9fd057b1680d001de38802ce1c28d79f04,5e03d22d6cb5bfdaf941b7749e0a313bd6ab2f86,1,2,1,1,3 days 04:15:10,2023-06-02T05:07:40Z,2023-06-05T09:22:50Z,500.0,...,262.0,304.0,304.0,501.0,262.0,262.0,304.0,693.0,274510000000000,1
2,84be0f9fd057b1680d001de38802ce1c28d79f04,e0d8519a9d7b6dfac29c1daf4199b6fe87781d0a,1,36,67,9,0 days 05:24:03,2023-06-02T03:16:04Z,2023-06-02T08:40:07Z,500.0,...,262.0,304.0,304.0,499.0,261.0,261.0,304.0,693.0,19443000000000,1
3,84be0f9fd057b1680d001de38802ce1c28d79f04,d0a47b9811a30f5de6d59334bb4fcc8c74ef640b,1,21,2,3,4 days 00:22:26,2023-06-01T08:30:26Z,2023-06-05T08:52:52Z,500.0,...,262.0,304.0,304.0,500.0,261.0,261.0,304.0,693.0,346946000000000,1
4,1b918160214a0b20ce1ae8f9e02f8e0dce243b53,988d2f0c4ef3ba8db0e701ab2105555996def4f7,8,211,212,80,12 days 00:39:27,2023-06-01T07:45:26Z,2023-06-13T08:24:53Z,499.0,...,261.0,304.0,304.0,500.0,262.0,262.0,304.0,693.0,1039167000000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
493,15636bdea1580196118b942d2d3587e0262d2b8f,044f62ef32cd0d195bc4bb6c8492d48d15379ff4,9,412,287,12,1 days 16:30:23,2022-09-28T17:53:03Z,2022-09-30T10:23:26Z,241.0,...,129.0,139.0,139.0,290.0,175.0,175.0,194.0,497.0,145823000000000,1
494,05d105dea3ee53b9d001adbfcf2ec87ea06f8aba,8fd966394b66ef83dca4635a9c278f655903a9bf,16,478,110,19,9 days 11:33:16,2022-09-28T15:15:23Z,2022-10-08T02:48:39Z,290.0,...,175.0,194.0,194.0,290.0,175.0,175.0,194.0,500.0,819196000000000,1
517,41751d8b636064d73ffb66ab07cea0c7e3091e47,acf4d5b1b469adf383b06266ef2e9815f472cd25,15,1601,1,22,110 days 16:44:58,2022-09-21T09:21:12Z,2023-01-10T02:06:10Z,384.0,...,213.0,238.0,238.0,386.0,214.0,214.0,243.0,546.0,9564298000000000,1
524,4969ffae88494b69de694f03a9c534f25dd5d242,38b0daf8b7ba723f2ee18d1e0ab55073701c1b74,39,1164,97,22,32 days 19:01:36,2022-09-18T11:31:28Z,2022-10-21T06:33:04Z,299.0,...,176.0,163.0,163.0,303.0,183.0,183.0,180.0,484.0,2833296000000000,1


In [83]:
X.columns[:3]

Index(['commits', 'additions', 'deletions'], dtype='object')

In [92]:
results_df

,combination,cluster_num,silhouette_score,cluster_sort,time_min,time_max,time_-2std,time_-1std,time_+1std,time_+2std
0,(),0,0.729828,2,0 days 00:00:05,110 days 16:44:58,-17 days +13:23:06.703155716,-6 days +10:01:23.543885550,16 days 03:17:57.225345218,26 days 23:56:14.066075052
1,(),1,0.729828,2,0 days 00:00:06,41 days 21:36:58,-7 days +03:08:01.512097444,-3 days +15:43:57.795247328,6 days 16:55:50.361547096,11 days 05:31:46.644696980
2,"(commits,)",0,0.729828,2,0 days 00:00:05,110 days 16:44:58,-17 days +13:23:06.703155716,-6 days +10:01:23.543885550,16 days 03:17:57.225345218,26 days 23:56:14.066075052
3,"(commits,)",1,0.729828,2,0 days 00:00:06,41 days 21:36:58,-7 days +03:08:01.512097444,-3 days +15:43:57.795247328,6 days 16:55:50.361547096,11 days 05:31:46.644696980
4,"(additions,)",0,0.729828,2,0 days 00:00:05,110 days 16:44:58,-17 days +13:23:06.703155716,-6 days +10:01:23.543885550,16 days 03:17:57.225345218,26 days 23:56:14.066075052
...,...,...,...,...,...,...,...,...,...,...
32763,"(commits, deletions, changed_files, begin_Disp...",1,0.729828,2,0 days 00:00:06,41 days 21:36:58,-7 days +03:08:01.512097444,-3 days +15:43:57.795247328,6 days 16:55:50.361547096,11 days 05:31:46.644696980
32764,"(additions, deletions, changed_files, begin_Di...",0,0.729828,2,0 days 00:00:05,110 days 16:44:58,-17 days +13:23:06.703155716,-6 days +10:01:23.543885550,16 days 03:17:57.225345218,26 days 23:56:14.066075052
32765,"(additions, deletions, changed_files, begin_Di...",1,0.729828,2,0 days 00:00:06,41 days 21:36:58,-7 days +03:08:01.512097444,-3 days +15:43:57.795247328,6 days 16:55:50.361547096,11 days 05:31:46.644696980
32766,"(commits, additions, deletions, changed_files,...",0,0.729828,2,0 days 00:00:05,110 days 16:44:58,-17 days +13:23:06.703155716,-6 days +10:01:23.543885550,16 days 03:17:57.225345218,26 days 23:56:14.066075052
